In [114]:
import wbdata

In [115]:
wb_regions = ['Arab World', 'Caribbean small states',
              'Central Europe and the Baltics', 'Early-demographic dividend',
              'East Asia & Pacific', 'East Asia & Pacific (excluding high income)',
              'East Asia & Pacific (IDA & IBRD countries)',
              'Euro area', 'Europe & Central Asia',
              'Europe & Central Asia (excluding high income)',
              'Europe & Central Asia (IDA & IBRD countries)',
              'European Union',
              'Fragile and conflict affected situations',
              'Heavily indebted poor countries (HIPC)',
              'High income',
              'IBRD only',
              'IDA & IBRD total',
              'IDA blend',
              'IDA only',
              'IDA total',
              'Late-demographic dividend',
              'Latin America & Caribbean',
              'Latin America & Caribbean (excluding high income)',
              'Latin America & the Caribbean (IDA & IBRD countries)',
              'Least developed countries',
              'Least developed countries: UN classification',
              'Low & middle income',
              'Low income',
              'Lower middle income',
              'Middle East & North Africa',
              'Middle East & North Africa (excluding high income)',
              'Middle East & North Africa (IDA & IBRD countries)',
              'Middle income',
              'North America',
              'Not classified',
              'OECD members',
              'Other small states',
              'Pacific island small states',
              'Post-demographic dividend',
              'Pre-demographic dividend',
              'Small states',
              'South Asia',
              'South Asia (IDA & IBRD)',
              'Sub-Saharan Africa',
              'Sub-Saharan Africa (excluding high income)',
              'Sub-Saharan Africa (IDA & IBRD countries)',
              'Upper middle income',
              'World']

In [116]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [117]:
Governmentdebt_raw = wbdata.get_data("GC.DOD.TOTL.GD.ZS")

In [118]:
Inflation_raw = wbdata.get_data("FP.CPI.TOTL.ZG")

In [119]:
Expense_raw = wbdata.get_data("GC.XPN.TOTL.GD.ZS")

In [120]:
Tax_raw = wbdata.get_data("GC.TAX.TOTL.GD.ZS")

In [121]:
Reserves_raw = wbdata.get_data("FI.RES.TOTL.CD")

In [122]:
Officialexchange_raw = wbdata.get_data("PA.NUS.FCRF")

In [123]:
def wb_clean_year_data(wb_raw, year):
    rv = {}
    for X in wb_raw:
        date = X['date']
        country = X['country']['value']
        if date == year and country not in wb_regions:
            try:
                value = float(X['value'])
                rv[country] = value
            except:
                pass
    return rv

In [124]:
Governmentdebt_clean = wb_clean_year_data(Governmentdebt_raw, '2015')

In [127]:
Governmentdebt_df = pd.DataFrame({'pais': list(Governmentdebt_clean.keys()), 'Gov.Debt_2015': list(Governmentdebt_clean.values())})

In [128]:
Governmentdebt_df

,pais,Gov.Debt_2015
0,Albania,79.864445
1,Australia,0.046948
2,"Bahamas, The",48.119517
3,Barbados,141.413895
4,Belarus,37.598178
5,Bhutan,93.969228
6,Botswana,17.909159
7,Brazil,67.537554
8,Colombia,66.667819
9,El Salvador,51.278928


In [129]:
def wb_dict_to_dataframe(dict_clean, nombre_variable, año):
    df = pd.DataFrame({'pais': list(dict_clean.keys()),
                       nombre_variable: list(dict_clean.values()),
                       'año': [año] * len(dict_clean)})
    return df

In [130]:
variables=['GC.DOD.TOTL.GD.ZS','FP.CPI.TOTL.ZG','GC.XPN.TOTL.GD.ZS','GC.TAX.TOTL.GD.ZS','FI.RES.TOTL.CD','PA.NUS.FCRF']

In [131]:
Governmentdebt_raw = wbdata.get_data("GC.DOD.TOTL.GD.ZS")
Inflation_raw = wbdata.get_data("FP.CPI.TOTL.ZG")
Expense_raw = wbdata.get_data("GC.XPN.TOTL.GD.ZS")
Tax_raw = wbdata.get_data("GC.TAX.TOTL.GD.ZS")
Reserves_raw = wbdata.get_data("FI.RES.TOTL.CD")
Officialexchange_raw = wbdata.get_data("PA.NUS.FCRF")

In [132]:
raw_lists = {'Deuda': Governmentdebt_raw,
             'Inflacion': Inflation_raw,
             'Gasto público': Expense_raw,
             'Tasa fiscal': Tax_raw,
             'Reservas':Reserves_raw,
             'Tipo de cambio':Officialexchange_raw}
            

In [139]:
for año in range(1985, 2015):
    dfs_año = []
    for k, v in raw_lists.items():
        v_clean = wb_clean_year_data(v, str(año))
        df = wb_dict_to_dataframe(v_clean, k, str(año))
        dfs_año.append(df)
    if año == 1989:
        for i, df in enumerate(dfs_año):
            if i == 0:
                df_año = df.merge(dfs_año[i + 1], on=['pais', 'año'])
            else:
                try:
                    df_año = df_año.merge(dfs_año[i + 1], on=['pais', 'año'])
                except:
                    pass
    else:
        for i, df in enumerate(dfs_año):
            if i == 0:
                df_otros_años = df.merge(dfs_año[i + 1], on=['pais', 'año'])
            else:
                try:
                    df_otros_años = df_otros_años.merge(dfs_año[i + 1], on=['pais', 'año'])
                except:
                    pass
        df_año = df_año.append(df_otros_años, ignore_index=True,)

In [140]:
df_año.to_excel('base_ecoproy.xlsx')

In [141]:
df_año.head (92) 

,pais,Deuda,año,Inflacion,Gasto público,Tasa fiscal,Reservas,Tipo de cambio
0,United States,39.128400,1989,4.827003,20.863127,10.782440,1.685836e+11,1.000000
1,Australia,12.253160,1990,7.333022,21.861559,22.476879,1.931874e+10,1.281057
2,"Bahamas, The",24.418825,1990,4.669319,16.610865,13.862919,1.581654e+08,1.000000
3,Bahrain,8.727364,1990,0.929335,27.118964,5.187374,1.292613e+09,0.376000
4,Bhutan,51.188019,1990,10.000000,20.557124,4.413943,8.883421e+07,17.505325
...,...,...,...,...,...,...,...,...
87,Bhutan,45.480489,1992,15.980114,18.196810,5.048773,8.512000e+07,25.918083
88,Botswana,12.532151,1992,16.167612,30.498748,27.845540,3.793419e+09,2.109725
89,Burundi,103.991580,1992,1.823333,27.691933,15.122784,1.799118e+08,208.302667
90,Cyprus,95.833990,1992,6.510220,50.698586,30.868149,1.180838e+09,0.449549


In [142]:
df_año ['pais'].value_counts()

United States           26
Switzerland             25
United Kingdom          25
Jordan                  25
Singapore               25
                        ..
Egypt, Arab Rep.         1
United Arab Emirates     1
Kyrgyz Republic          1
Poland                   1
Ireland                  1
Name: pais, Length: 91, dtype: int64

In [147]:
missing_years = []
for c in df_año['pais'].unique():
    c_df = df_año.loc[df_año.loc['pais'] == c]
    for y in df_año['año'].unique():
        c_y_df = c_df.loc[c_df['año'] == y]
        if len(c_y_df) == 0:
            print(c, y)
            missing_years.append(y)

KeyError: 'pais'